# Análise bioinformática da proteína da cápside codificada pelo GeneID 1258751

## Importação de bibliotecas

Nesta secção são importadas as bibliotecas necessárias para a análise bioinformática nomeadamente módulos do BioPython para acesso às bases de dados do NCBI, manipulação de sequências, alinhamentos múltiplos e construção de árvores filogenéticas.

In [25]:
from Bio import Entrez, SeqIO, AlignIO, Phylo
from Bio.Blast import NCBIWWW, NCBIXML
from Bio.Phylo.TreeConstruction import DistanceCalculator, DistanceTreeConstructor

Entrez.email = "dmartins0080@gmail.com"
gene_id = "1258751"

## Obtenção da sequência de aminoácidos

É utilizada a base de dados NCBI para obter a sequência proteica associada ao GeneID 1258751. A sequência é descarregada em formato FASTA e utilizada como base para as análises subsequentes.


In [35]:
handle = Entrez.efetch(db="gene", id=gene_id, rettype="xml")
gene_record = Entrez.read(handle)
handle.close()

gene = gene_record[0]

capside_id = gene["Entrezgene_locus"][0]["Gene-commentary_products"][0]["Gene-commentary_accession"]
print("Proteína associada:", capside_id)

handle = Entrez.efetch(db="protein", id=capside_id, rettype="fasta", retmode="text")
protein_record = SeqIO.read(handle, "fasta")
handle.close()

print("Descrição da proteína:" , (protein_record.description))
print("Comprimento:", len(protein_record.seq))


Proteína associada: NP_049787
Descrição da proteína: NP_049787.1 major head protein [Escherichia phage T4]
Comprimento: 521


## Realização de um BLASTp


A sequência proteica obtida é analisada por BLASTp contra a base de dados nr (non-redundant), com o objetivo de confirmar a sua identidade funcional e identificar proteínas homólogas. Os 10 melhores alinhamentos são utilizados nos passos seguintes.

In [ ]:
SeqIO.write(protein_record,"gene1258751_protein.fasta","fasta")
with open("gene1258751_protein.fasta") as seq_fasta:
    fasta_seq = seq_fasta.read()

result_handle = NCBIWWW.qblast(program="blastp", database="nr", sequence=fasta_seq)

with open("gene1258751_blast.xml", "w") as out:
    out.write(result_handle.read())

with open("gene1258751_blast.xml") as handle:
    blast_record = NCBIXML.read(handle)

print("Top 10 hits BLAST:")
for i, alignment in enumerate(blast_record.alignments[:10], start=1):
    hsp = alignment.hsps[0]
    print(f"{alignment.accession}")
    print("   Descrição:", alignment.hit_def)
    print("   Identidade:", hsp.identities, "/", hsp.align_length)
    print("   E-value:", hsp.expect)


A correr BLASTp (isto pode demorar)...
BLAST terminado. Resultado guardado em gene1258751_blast.xml


## Alinhamento múltiplo de sequências

As sequências homólogas selecionadas são alinhadas utilizando o programa Clustal Omega, permitindo avaliar a conservação da sequência proteica entre diferentes organismos.

In [ ]:

with open("gene1258751_blast.xml") as handle:
    blast_record = NCBIXML.read(handle)

hit_accessions = []
records=[]
for alignment in blast_record.alignments:
    hit_accessions.append(alignment.accession)
    if len(hit_accessions) == 10:
        break

print("Accessions usados:")
for acc in hit_accessions:
    print(' ',(acc))

for acc in hit_accessions:
    handle = Entrez.efetch(db="protein", id=acc,rettype="fasta", retmode="text")
    record = SeqIO.read(handle, "fasta")
    handle.close()
    records.append(record)

SeqIO.write(records, "capsid_hits.fasta", "fasta")



Accessions usados:
  NP_049787
  WP_371335560
  WP_411401423
  YP_010074593
  UJD20117
  YP_010075713
  YP_010072737
  YP_010066370
  EOQ0467934
  UGO56632


10

Corrido no docker: 

docker run --rm -v ${PWD}:/data biocontainers/clustalo:v1.2.4-2-deb_cv1 clustalo -i /data/capsid_hits.fasta -o /data/capsid_alignment.aln --force

## Construção da árvore filogenética

A partir do alinhamento múltiplo é construída uma árvore filogenética pelo método Neighbor-Joining, tendo como base uma matriz de disntâncias de identidade, de forma a visualizar as relações evolutivas entre a proteína de estudo e aquelas que foram usadas no alinhamento.


In [34]:
alinhamento = AlignIO.read("capsid_alignment.aln", "clustal")

calculator = DistanceCalculator("identity")
distancias = calculator.get_distance(alinhamento)

id_para_nome = {}

for registo in SeqIO.parse("capsid_hits.fasta", "fasta"):
    id_para_nome[registo.id] = registo.description

constructor = DistanceTreeConstructor()
arvore = constructor.nj(distancias)

for folha in arvore.get_terminals():
    if folha.name in id_para_nome:
        folha.name = id_para_nome[folha.name]

Phylo.draw_ascii(arvore)


Phylo.write(arvore, "arvore_capside_completa.nwk", "newick")



  _______ NP_049787.1 major head protein [Esche...
 |
 |        ________________ YP_010072737.1 major head protein [Es...
 |       |
 |_______|         _______ YP_010075713.1 major head protein [Se...
 |       |        |
 |       |________|        _______ EOQ0467934.1 phage capsid protein [Es...
 |                |  _____|
 |                | |     | ________ UGO56632.1 putative major capsid prot...
 |                |_|     ||
 |                  |      | YP_010066370.1 major head protein [En...
_|                  |
 |                  |_____ YP_010074593.1 major head protein [Es...
 |
 | WP_371335560.1 MULTISPECIES: phage ca...
 |
 |_______ WP_411401423.1 hypothetical protein [...
 |
 |_______ UJD20117.1 major head protein [Escher...



1